In [3]:
import java.io.File
abstract class Problem<T>(private val filename: String) {
    abstract val testResult: T
    protected var isTesting: Boolean = true
        private set
    fun run() {
        val testOutput = File("testData/$filename").useLines { solve(it.toList()) }
        if (testOutput != testResult) {
            throw IllegalStateException("Test output doesn't match expected result \n Actual: $testOutput \n Expected: $testResult")
        }
        println("Test results: Expected=$testResult Actual=$testOutput")

        isTesting = false
        val actualOutput = File("data/$filename").useLines { solve(it.toList()) }
        println("Solution: $actualOutput")
    }
    abstract fun solve(lines: List<String>): T
}

## Day 6: Cephalopod math
### Part 1
After helping the Elves in the kitchen, you were taking a break and helping them re-enact a movie scene when you over-enthusiastically jumped into the garbage chute!

A brief fall later, you find yourself in a garbage smasher. Unfortunately, the door's been magnetically sealed.

As you try to find a way out, you are approached by a family of cephalopods! They're pretty sure they can get the door open, but it will take some time. While you wait, they're curious if you can help the youngest cephalopod with her math homework.

Cephalopod math doesn't look that different from normal math. The math worksheet (your puzzle input) consists of a list of problems; each problem has a group of numbers that need to be either added (`+`) or multiplied (`*`) together.

However, the problems are arranged a little strangely; they seem to be presented next to each other in a very long horizontal list. For example:
```
123 328  51 64
 45 64  387 23
  6 98  215 314
*   +   *   +
```
Each problem's numbers are arranged vertically; at the bottom of the problem is the symbol for the operation that needs to be performed. Problems are separated by a full column of only spaces. The left/right alignment of numbers within each problem can be ignored.

So, this worksheet contains four problems:
```
123 * 45 * 6 = 33210
328 + 64 + 98 = 490
51 * 387 * 215 = 4243455
64 + 23 + 314 = 401
```
To check their work, cephalopod students are given the grand total of adding together all of the answers to the individual problems. In this worksheet, the grand total is `33210 + 490 + 4243455 + 401 = 4277556`.

Of course, the actual worksheet is much wider. You'll need to make sure to unroll it completely so that you can read the problems clearly.

Solve the problems on the math worksheet. What is the grand total found by adding together all of the answers to the individual problems?

In [4]:
class DaySixPartOne(override val testResult: Long = 4277556) : Problem<Long>("day6.txt") {
    override fun solve(lines: List<String>): Long {
        val data = lines.map { it.trim().split("\\s+".toRegex()) }
        return data.rotate().sumOf { row ->
            return@sumOf when (row.first()) {
                "+" -> row.subList(1, row.size).sumOf { it.toLong() }
                "*" -> {
                    row.subList(1, row.size).map { it.toLong() }.product()
                }
                else -> 0L
            }
        }
    }

    private fun <T> List<List<T>>.rotate(): List<List<T>> {
        val numCols = this[0].size
        val numRows = this.size

        val rotated = List(numCols) { mutableListOf<T>() }

        for (i in 0 until numRows) {
            for (j in 0 until numCols) {
                rotated[j].add(0, this[i][j])
            }
        }

        return rotated.map { it.toList() }
    }

    private fun List<Long>.product(): Long {
        var out = this.first()
        this.subList(1, this.size).forEach { out *= it }
        return out
    }
}

DaySixPartOne().run()

Test results: Expected=4277556 Actual=4277556
Solution: 4309240495780


### Part 2
The big cephalopods come back to check on how things are going. When they see that your grand total doesn't match the one expected by the worksheet, they realize they forgot to explain how to read cephalopod math.

Cephalopod math is written right-to-left in columns. Each number is given in its own column, with the most significant digit at the top and the least significant digit at the bottom. (Problems are still separated with a column consisting only of spaces, and the symbol at the bottom of the problem is still the operator to use.)

Here's the example worksheet again:
```
123 328  51 64
 45 64  387 23
  6 98  215 314
*   +   *   +
```
Reading the problems right-to-left one column at a time, the problems are now quite different:

The rightmost problem is `4 + 431 + 623 = 1058`
The second problem from the right is `175 * 581 * 32 = 3253600`
The third problem from the right is `8 + 248 + 369 = 625`
Finally, the leftmost problem is `356 * 24 * 1 = 8544`
Now, the grand total is `1058 + 3253600 + 625 + 8544 = 3263827`.

Solve the problems on the math worksheet again. What is the grand total found by adding together all of the answers to the individual problems?

In [5]:
class DaySixPartTwo(override val testResult: Long = 3263827) : Problem<Long>("day6.txt") {
    override fun solve(lines: List<String>): Long {
        val longest = lines.maxOf { it.length }
        val data = lines.map { line ->
            var out = line.toMutableList()
            repeat(longest-line.length) {
                out.add(' ')
            }
            out.reversed().joinToString("")
        }

        val noOp = data.dropLast(1)
        val operators = data.takeLast(1)

        var sum = 0L
        val numbers = mutableListOf<Long>()
        for (col in 0..data.first().length - 1) {
            val num = noOp.map { it[col] }.joinToString("").trim()
            if (num.isNotBlank()) numbers.add(num.toLong())
            when (operators[0].getOrNull(col)) {
                '+' -> {
                    sum += numbers.sum()
                    numbers.clear()
                }
                '*' -> {
                    sum += numbers.product()
                    numbers.clear()
                }
            }
        }
        return sum
    }

    private fun List<Long>.product(): Long {
        var product = 1L
        this.forEach { product *= it }
        return product
    }
}

DaySixPartTwo().run()

Test results: Expected=3263827 Actual=3263827
Solution: 9170286552289
